In [1]:
import pandas as pd
import numpy as np
import dask.array as da
import dask.dataframe as dd
import time
import math

from netCDF4 import Dataset
import os,datetime,sys,fnmatch
import h5py

In [2]:
from dask.distributed import Client, LocalCluster
cluster = LocalCluster()
client = Client(cluster)

/Users/dprakas1/anaconda3/lib/python3.7/site-packages/distributed/dashboard/core.py:74: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn("\n" + msg)


In [3]:
client

Client Scheduler: tcp://127.0.0.1:60881 Dashboard: http://127.0.0.1:60882/status,Cluster Workers: 4 Cores: 4 Memory: 17.18 GB


In [4]:
%%time
def read_filelist(loc_dir,prefix,unie,fileformat):
    # Read the filelist in the specific directory
    str = os.popen("ls "+ loc_dir + prefix + unie + "*."+fileformat).read()
    fname = np.array(str.split("\n"))
    fname = np.delete(fname,len(fname)-1)
    
    return fname


def read_MODIS(fname1,fname2,verbose=False): # READ THE HDF FILE
    # Read the cloud mask from MYD06_L2 product')
    ncfile=Dataset(fname1,'r')
    CM1km = np.array(ncfile.variables['Cloud_Mask_1km'])
    CM   = (np.array(CM1km[:,:,0],dtype='byte') & 0b00000110) >>1
    #ncfile = Dataset(fname1, "r")
    #CM = myd06.variables["Cloud_Mask_1km"][:,:,:] # Reading Specific Variable 'Cloud_Mask_1km'.
    #CM   = (np.array(CM[:,:,0],dtype='byte') & 0b00000110) >>1
    ncfile.close()
    
    ncfile=Dataset(fname2,'r')
    lat  = np.array(ncfile.variables['Latitude'])
    lon  = np.array(ncfile.variables['Longitude'])
    #ncfile = Dataset(MOD03_files, "r")
    #latitude = myd03.variables["Latitude"][:,:] # Reading Specific Variable 'Latitude'.
    #latitude = np.array(latitude).byteswap().newbyteorder() # Addressing Byteswap For Big Endian Error.
    #longitude = myd03.variables["Longitude"][:,:] # Reading Specific Variable 'Longitude'.
    attr_lat = ncfile.variables['Latitude']._FillValue
    attr_lon = ncfile.variables['Longitude']._FillValue
    return lat,lon,CM


def countzero(x, axis=1):
    #print(x)
    count0 = 0
    count1 = 0
    for i in x:
        if i <= 1:
            count0 +=1
    #print(count0/len(x))
    return count0/len(x)

satellite = 'Aqua'

MYD06_dir= '/Users/dprakas1/Desktop/modis_files/'
MYD06_prefix = 'MYD06_L2.A2008'
MYD03_dir= '/Users/dprakas1/Desktop/modis_files/'
MYD03_prefix = 'MYD03.A2008'
fileformat = 'hdf'

fname1,fname2 = [],[]


days = np.arange(1,31,dtype=np.int)
for day in days:
    dc ='%03i' % day
    fname_tmp1 = read_filelist(MYD06_dir,MYD06_prefix,dc,fileformat)
    fname_tmp2 = read_filelist(MYD03_dir,MYD03_prefix,dc,fileformat)
    fname1 = np.append(fname1,fname_tmp1)
    fname2 = np.append(fname2,fname_tmp2)

# Initiate the number of day and total cloud fraction
files  = np.arange(len(fname1))



for j in range(0,1):#hdfs:
    print('steps: ',j+1,'/ ',(fname1)) 

    # Read Level-2 MODIS data
    lat,lon,CM = read_MODIS(fname1[j],fname2[j])
print((fname1))
print((fname2))
#rint(CM)
#lat = lat.ravel()
#lon = lon.ravel()
#CM  = CM.ravel()
CM.shape    



cm = np.zeros((2030,1354), dtype=np.float32)

for MOD06_file in fname1:
    #print(MOD06_file)
    myd06 = Dataset(MOD06_file, "r")
    CM = myd06.variables["Cloud_Mask_1km"][:,:,0]# Reading Specific Variable 'Cloud_Mask_1km'.
    CM = (np.array(CM,dtype='byte') & 0b00000110) >>1
    CM = np.array(CM).byteswap().newbyteorder()
    #cm = da.from_array(CM, chunks =(2030,1354))
    #print(CM.shape)
    #cm = np.concatenate((cm,CM))
    cm = da.concatenate((cm,CM),axis=0)
    #bit0 = np.dstack((bit0,bit0r))
    #bit12 = np.dstack((bit12,bit12r))
    
print('The Cloud Mask Array Shape Is: ',cm.shape)


lat = np.zeros((2030,1354), dtype=np.float32)
lon = np.zeros((2030,1354), dtype=np.float32)
for MOD03_file in fname2:
    #print(MOD03_file)
    myd03 = Dataset(MOD03_file, "r")
    latitude = myd03.variables["Latitude"][:,:]# Reading Specific Variable 'Latitude'.
    #lat = da.from_array(latitude, chunks =(2030,1354))
    lat = da.concatenate((lat,latitude),axis=0)


    longitude = myd03.variables["Longitude"][:,:] # Reading Specific Variable 'Longitude'.
    #lon = da.from_array(longitude, chunks =(2030,1354))
    lon = da.concatenate((lon,longitude),axis=0)
    
print('Longitude Shape Is: ',lon.shape)
print('Latitude Shape Is: ',lat.shape)

cm=da.ravel(cm)
lat=da.ravel(lat)
lon=da.ravel(lon)

lon=lon.astype(int)
lat=lat.astype(int)
cm=cm.astype(int)


Lat=lat.to_dask_dataframe()
Lon=lon.to_dask_dataframe()
CM=cm.to_dask_dataframe()

df=dd.concat([Lat,Lon,CM],axis=1,interleave_partitions=False)

cols = {0:'Latitude',1:'Longitude',2:'CM'}
df = df.rename(columns=cols)
#df.compute()

df2=df.groupby(['Longitude','Latitude']).CM.apply(countzero).reset_index()

df3=df2.compute(num_workers=4)

combs=[]
for x in range(-89,91):
    for y in range(-179,181):
        combs.append((x, y))
        
df_1=pd.DataFrame(combs)
df_1.columns=['Latitude','Longitude']
df_2=dd.from_pandas(df_1,npartitions=1)

df4=pd.merge(df_1, df3,on=('Longitude','Latitude'), how='left')

df5=df4['CM'].values

b=df5.reshape(180,360)

print(b)







steps:  1 /  ['/Users/dprakas1/Desktop/modis_files/MYD06_L2.A2008001.0000.006.2013341193524.hdf'
 '/Users/dprakas1/Desktop/modis_files/MYD06_L2.A2008001.0005.006.2013341193207.hdf'
 '/Users/dprakas1/Desktop/modis_files/MYD06_L2.A2008001.0010.006.2013341192125.hdf']
['/Users/dprakas1/Desktop/modis_files/MYD06_L2.A2008001.0000.006.2013341193524.hdf'
 '/Users/dprakas1/Desktop/modis_files/MYD06_L2.A2008001.0005.006.2013341193207.hdf'
 '/Users/dprakas1/Desktop/modis_files/MYD06_L2.A2008001.0010.006.2013341192125.hdf']
['/Users/dprakas1/Desktop/modis_files/MYD03.A2008001.0000.006.2012066122450.hdf'
 '/Users/dprakas1/Desktop/modis_files/MYD03.A2008001.0005.006.2012066122516.hdf'
 '/Users/dprakas1/Desktop/modis_files/MYD03.A2008001.0010.006.2012066122416.hdf']
The Cloud Mask Array Shape Is:  (8120, 1354)
Longitude Shape Is:  (8120, 1354)
Latitude Shape Is:  (8120, 1354)


/Users/dprakas1/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:135: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result


[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]
CPU times: user 5.39 s, sys: 480 ms, total: 5.87 s
Wall time: 9.29 s


In [4]:
%%time
import pandas as pd
import numpy as np
import dask.array as da
import dask.dataframe as dd
import dask.delayed as delayed
import time
import math
#import graphviz
from netCDF4 import Dataset
import os,datetime,sys,fnmatch
import h5py
import dask

def read_filelist(loc_dir,prefix,unie,fileformat):
    # Read the filelist in the specific directory
    str = os.popen("ls "+ loc_dir + prefix + unie + "*."+fileformat).read()
    fname = np.array(str.split("\n"))
    fname = np.delete(fname,len(fname)-1)
    
    return fname

def read_MODIS(fname1,fname2,verbose=False): # READ THE HDF FILE
    # Read the cloud mask from MYD06_L2 product')
    ncfile=Dataset(fname1,'r')
    CM1km = np.array(ncfile.variables['Cloud_Mask_1km'])
    CM   = (np.array(CM1km[:,:,0],dtype='byte') & 0b00000110) >>1
    #ncfile = Dataset(fname1, "r")
    #CM = myd06.variables["Cloud_Mask_1km"][:,:,:] # Reading Specific Variable 'Cloud_Mask_1km'.
    #CM   = (np.array(CM[:,:,0],dtype='byte') & 0b00000110) >>1
    CM=delayed(CM)
    ncfile.close()
    
    ncfile=Dataset(fname2,'r')
    lat  = np.array(ncfile.variables['Latitude'])
    lon  = np.array(ncfile.variables['Longitude'])
    #ncfile = Dataset(MOD03_files, "r")
    #latitude = myd03.variables["Latitude"][:,:] # Reading Specific Variable 'Latitude'.
    #latitude = np.array(latitude).byteswap().newbyteorder() # Addressing Byteswap For Big Endian Error.
    #longitude = myd03.variables["Longitude"][:,:] # Reading Specific Variable 'Longitude'.
    attr_lat = ncfile.variables['Latitude']._FillValue
    attr_lon = ncfile.variables['Longitude']._FillValue
    lat=delayed(lat)
    lon=delayed(lon)
    return lat,lon,CM


def countzero(x, axis=1):
    #print(x)
    count0 = 0
    count1 = 0
    for i in x:
        if i <= 1:
            count0 +=1
    #print(count0/len(x))
    return (count0/len(x))



MYD06_dir= '/Users/dprakas1/Desktop/modis_files/'
MYD06_prefix = 'MYD06_L2.A2008'
MYD03_dir= '/Users/dprakas1/Desktop/modis_files/'
MYD03_prefix = 'MYD03.A2008'
fileformat = 'hdf'

fname1,fname2 = [],[]


days = np.arange(1,31,dtype=np.int)
for day in days:
    dc ='%03i' % day
    fname_tmp1 = read_filelist(MYD06_dir,MYD06_prefix,dc,fileformat)
    fname_tmp2 = read_filelist(MYD03_dir,MYD03_prefix,dc,fileformat)
    fname1 = np.append(fname1,fname_tmp1)
    fname2 = np.append(fname2,fname_tmp2)

# Initiate the number of day and total cloud fraction
files  = np.arange(len(fname1))



for j in range(0,1):#hdfs:
    ('steps: ',j+1,'/ ',(fname1)) 

    # Read Level-2 MODIS data
    lat,lon,CM = read_MODIS(fname1[j],fname2[j])

#rint(CM)
lat = lat.compute()
lon = lon.compute()
CM  = CM.compute()




cloud_pix = np.zeros((180, 360))
delayed_b1=[]
def aggregateOneFileData(M06_file, M03_file):
    cm = np.zeros((2030,1354), dtype=np.float32)
    lat = np.zeros((2030,1354), dtype=np.float32)
    lon = np.zeros((2030,1354), dtype=np.float32)
    
    print(x,y)
    myd06 = Dataset(M06_file, "r")
    CM = myd06.variables["Cloud_Mask_1km"][:,:,0]# Reading Specific Variable 'Cloud_Mask_1km'.
    CM = (np.array(CM,dtype='byte') & 0b00000110) >>1
    CM = np.array(CM).byteswap().newbyteorder()
    
    print("CM intial shape:",CM.shape)
    cm = da.concatenate((cm,CM),axis=0)
    #print("CM shape after con:",cm.shape)
    cm=da.ravel(cm)
    print("cm shape after ravel:",cm.shape)
    myd03 = Dataset(M03_file, "r")
    latitude = myd03.variables["Latitude"][:,:]
    longitude = myd03.variables["Longitude"][:,:]
    print("Lat intial shape:",latitude.shape)
    print("lon intial shape:",longitude.shape)
    
    lat = da.concatenate((lat,latitude),axis=0)
    lon = da.concatenate((lon,longitude),axis=0)
    print("lat shape after con:",lat.shape)
    print("lon shape after con:",lon.shape)
    
    lat=da.ravel(lat)
    lon=da.ravel(lon)
    
    print("lat shape after ravel:",lat.shape)
    print("lon shape after ravel:",lon.shape)
    cm=cm.astype(int)
    lon=lon.astype(int)
    lat=lat.astype(int)
    
    Lat=(lat.to_dask_dataframe())
    Lon=(lon.to_dask_dataframe())
    CM=(cm.to_dask_dataframe())
    df=(dd.concat([Lat,Lon,CM],axis=1,interleave_partitions=False))
    print(type(df))
    
    cols = {0:'Latitude',1:'Longitude',2:'CM'}
    df = df.rename(columns=cols)
    
    df2=delayed(df.groupby(['Longitude','Latitude']).CM.apply(countzero).reset_index())
    print(type(df2))
    df3=df2.compute()
    print(type(df3))
    
    df4=[df2['Longitude'].values,df2['Latitude'].values,df2['CM'].values]
    print(type(df4))
    
    delayed_b1.append(df4)
    
    
    
    return delayed_b1
    
    
for x,y in zip(fname1,fname2):
    results = aggregateOneFileData(x,y)
    print(results)
    
cf = np.zeros((180,360))
cf[:]=np.nan
for i in range(len(delayed_b1)):
    cf[(delayed_b1[i][1].compute()-90),(180+delayed_b1[i][0].compute())] = delayed_b1[i][2].compute()
print(cf)
client.close()

/Users/dprakas1/Desktop/modis_files/MYD06_L2.A2008001.0000.006.2013341193524.hdf /Users/dprakas1/Desktop/modis_files/MYD03.A2008001.0000.006.2012066122450.hdf
CM intial shape: (2030, 1354)
cm shape after ravel: (5497240,)
Lat intial shape: (2030, 1354)
lon intial shape: (2030, 1354)
lat shape after con: (4060, 1354)
lon shape after con: (4060, 1354)
lat shape after ravel: (5497240,)
lon shape after ravel: (5497240,)
<class 'dask.dataframe.core.DataFrame'>
<class 'dask.delayed.Delayed'>


/Users/dprakas1/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:142: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result


<class 'pandas.core.frame.DataFrame'>
<class 'list'>
[[Delayed('getattr-37b45b3cb57af0c3ad3f8d7758d58304'), Delayed('getattr-7f30d1967e5db57c091da874a186c288'), Delayed('getattr-20711fea999bdd460e4e5952170bf080')]]
/Users/dprakas1/Desktop/modis_files/MYD06_L2.A2008001.0005.006.2013341193207.hdf /Users/dprakas1/Desktop/modis_files/MYD03.A2008001.0005.006.2012066122516.hdf
CM intial shape: (2030, 1354)
cm shape after ravel: (5497240,)
Lat intial shape: (2030, 1354)
lon intial shape: (2030, 1354)
lat shape after con: (4060, 1354)
lon shape after con: (4060, 1354)
lat shape after ravel: (5497240,)
lon shape after ravel: (5497240,)
<class 'dask.dataframe.core.DataFrame'>
<class 'dask.delayed.Delayed'>
<class 'pandas.core.frame.DataFrame'>
<class 'list'>
[[Delayed('getattr-37b45b3cb57af0c3ad3f8d7758d58304'), Delayed('getattr-7f30d1967e5db57c091da874a186c288'), Delayed('getattr-20711fea999bdd460e4e5952170bf080')], [Delayed('getattr-03ff3b8d1cc8137b548c0498c9bb60cf'), Delayed('getattr-351db54a